In [1]:
# notebook mainly intentioned for quick, one-time data formatting that doesn't require a script 

In [2]:
import pandas as pd
import numpy as np
import sqlite3

In [3]:
# load the scheduling data

schedule_2024 = pd.read_csv(r"C:\Users\jonat\OneDrive\projects\scrape_and_score\data\raw\2024_schedule.txt")
schedule_2024.head()

,Week,Day,Date,Time,Winner/tie,Unnamed: 5,Loser/tie,Unnamed: 7,PtsW,PtsL,YdsW,TOW,YdsL,TOL
0,1,Thu,2024-09-05,8:20PM,Kansas City Chiefs,NaN,Baltimore Ravens,boxscore,27.0,20.0,353.0,1.0,452.0,1.0
1,1,Fri,2024-09-06,8:15PM,Philadelphia Eagles,NaN,Green Bay Packers,boxscore,34.0,29.0,410.0,3.0,414.0,1.0
2,1,Sun,2024-09-08,1:00PM,Pittsburgh Steelers,@,Atlanta Falcons,boxscore,18.0,10.0,270.0,0.0,226.0,3.0
3,1,Sun,2024-09-08,1:00PM,Buffalo Bills,NaN,Arizona Cardinals,boxscore,34.0,28.0,352.0,1.0,270.0,1.0
4,1,Sun,2024-09-08,1:00PM,New Orleans Saints,NaN,Carolina Panthers,boxscore,47.0,10.0,379.0,1.0,193.0,3.0


In [4]:
# keep only relevant data
schedule_2024 = schedule_2024[["Week", "Winner/tie", "Unnamed: 5", "Loser/tie"]]
schedule_2024.head()

,Week,Winner/tie,Unnamed: 5,Loser/tie
0,1,Kansas City Chiefs,NaN,Baltimore Ravens
1,1,Philadelphia Eagles,NaN,Green Bay Packers
2,1,Pittsburgh Steelers,@,Atlanta Falcons
3,1,Buffalo Bills,NaN,Arizona Cardinals
4,1,New Orleans Saints,NaN,Carolina Panthers


In [5]:
schedule_2024.rename(columns = {'Unnamed: 5': 'Home'}, inplace = True)
schedule_2024['Home'] = schedule_2024['Home'].copy() != '@'
schedule_2024.tail()

,Week,Winner/tie,Home,Loser/tie
267,18,New York Giants,False,Philadelphia Eagles
268,18,Los Angeles Chargers,False,Las Vegas Raiders
269,18,Seattle Seahawks,False,Los Angeles Rams
270,18,San Francisco 49ers,False,Arizona Cardinals
271,18,Washington Commanders,False,Dallas Cowboys


In [6]:
# Creating two dataframes: one where each row represents the visitor's perspective, 
# and another for the home team's perspective
visitor_df = schedule_2024[['Week', 'Winner/tie', 'Loser/tie']].rename(columns={'Winner/tie': 'team', 'Loser/tie': 'opp'})
home_df = schedule_2024[['Week', 'Winner/tie', 'Loser/tie']].rename(columns={'Winner/tie': 'opp', 'Loser/tie': 'team'})

# Concatenate both dataframes to double the length, with each team having an entry with its opponent
schedule_2024 = pd.concat([visitor_df, home_df], ignore_index = True)
schedule_2024.head()

,Week,team,opp
0,1,Kansas City Chiefs,Baltimore Ravens
1,1,Philadelphia Eagles,Green Bay Packers
2,1,Pittsburgh Steelers,Atlanta Falcons
3,1,Buffalo Bills,Arizona Cardinals
4,1,New Orleans Saints,Carolina Panthers


In [7]:
# cache the schedule data
schedule_2024.to_csv(r"C:\Users\jonat\OneDrive\projects\scrape_and_score\data\processed\2024_schedule.txt")

In [14]:
# access the passing data database

database = r"C:\Users\jonat\OneDrive\projects\scrape_and_score\data\quarterback.db"
conn = sqlite3.connect(database)

gamelogs = pd.read_sql_query("SELECT * FROM gamelogs", conn)
defense_stats = pd.read_sql_query("SELECT * FROM defense_stats", conn )

conn.close()

gamelogs.head()

,qb_id,year,week,name,team,pass_yards,opponent,defense_id,opp_rank,weight,adjusted_yards
0,05de5e56,2013,1,geno smith.txt,New York Jets,256,Tampa Bay Buccaneers,f94be5ec,0,0,0
1,05de5e56,2013,2,geno smith.txt,New York Jets,214,New England Patriots,14bf73b4,0,0,0
2,05de5e56,2013,3,geno smith.txt,New York Jets,331,Buffalo Bills,f33be5c7,0,0,0
3,05de5e56,2013,4,geno smith.txt,New York Jets,289,Tennessee Titans,63826309,0,0,0
4,05de5e56,2013,5,geno smith.txt,New York Jets,199,Atlanta Falcons,d9a07016,0,0,0


In [22]:
defense_stats.tail()

,defense_id,team,year,pyds_allowed,rank,weight
635,82491b28,New Orleans Saints,2024,3391.0,27,0.910516
636,07ef4cf3,Minnesota Vikings,2024,3429.0,28,0.901461
637,c0bc18a7,Tampa Bay Buccaneers,2024,3468.0,29,0.883880
638,c93417a7,Baltimore Ravens,2024,3625.0,30,0.869407
639,1c644145,Jacksonville Jaguars,2024,3700.0,31,0.847660
